In [1]:
import requests 
import random
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime
from time import sleep

In [2]:
# Con esto podemos evitar que el navegador detecte 
#que estamos usando un software de automatización
options = Options()
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)

In [3]:
# Podemos guardar las cookies

options.add_argument('user-data-dir=/tmp/selenium_profile') 

In [4]:
from fake_useragent import UserAgent

user = UserAgent().random

print(user)

Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36


In [5]:
options.add_argument(f'user-agent={user}')

In [6]:
#estadisticas summary table para ligas con 20 equipos temporada 22/23
def stats_summary(driver, league)->pd.DataFrame:
    
    #object of ActionChains
    a = ActionChains(driver)
    #identify element: "Leagues"
    m = driver.find_element_by_link_text(league)
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    print("Page title: " + driver.title)
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(9.0, 12.0))
    
    
    #change to a different season
    #object of ActionChains
    a = ActionChains(driver)
    #identify foldable element: "season"
    m = driver.find_element(By.ID, "seasons")
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    #click on a specific season 
    sleep(5)
    try:
        driver.find_element(By.CSS_SELECTOR, '#seasons > option:nth-child(2)').click()
    except:
        pass
     
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(9.0, 12.0))
    
    print("Page title: " + driver.title)
    
    if league == "Serie A":
        #change to the correct championship
        #object of ActionChains
        a = ActionChains(driver)
        #identify foldable element: "season"
        m = driver.find_element(By.ID, "stages")
        #hover over element to chose the specific league
        a.move_to_element(m).click().perform()
        #click on a specific season 
        sleep(5)
        try:
            driver.find_element(By.XPATH, '//*[@id="stages"]/option[1]').click()
            
        except:
            pass
    else:
        pass

    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(13.0, 15.0))
    
    a = ActionChains(driver)
    #identify element: "Leagues"
    s = driver.find_element_by_link_text("Team Statistics")
    #hover over element to chose the specific league
    a.move_to_element(s).click().perform()
    print("Page title: " + driver.title)
    
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(13.0, 15.0))
    
    table_data = []
    
    #summary table for team statistics
    cols = [e.text for e in driver.find_elements(By.TAG_NAME, 'th')[0:8]]
    cols.insert(3, "yellow")
    
    team = [i.find_element(By.TAG_NAME, 'a').text for i in driver.find_elements(By.CSS_SELECTOR, 'td.col12-lg-2.col12-m-3.col12-s-4.col12-xs-5.grid-abs.overflow-text')]
    link = [i.find_element(By.TAG_NAME, 'a').get_attribute('href') for i in driver.find_elements(By.CSS_SELECTOR, 'td.col12-lg-2.col12-m-3.col12-s-4.col12-xs-5.grid-abs.overflow-text')]

    goal = [i.text for i in driver.find_elements(By.CLASS_NAME, 'goal') if i.text.isdigit()]
    shotspergame = [i.text for i in driver.find_elements(By.CLASS_NAME, 'shotsPerGame')[1:]]
    yellow = [i.text for i in driver.find_elements(By.CLASS_NAME, 'yellow-card-box') if i.text.isdigit()]
    red = [i.text for i in driver.find_elements(By.CLASS_NAME, 'red-card-box') if i.text.isdigit()]
    possession = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.possession')]
    passes = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.passSuccess')]
    aerialswon = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.aerialWonPerGame')]
    rating = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.sorted')[0:20]]
    
    table_data.append([team, goal, shotspergame, yellow, red, possession, passes, aerialswon, rating]) 
    
    table = {col : data for col, data in zip(cols, table_data[0])}
    summary_df = pd.DataFrame(table)
    sleep(3)
    
    return summary_df
    

In [74]:
chrome_driver = "./chromedriver.exe"
driver = webdriver.Chrome(executable_path = chrome_driver)
driver.get("https://www.whoscored.com/")
try:
    driver.find_element(By.CSS_SELECTOR, '#qc-cmp2-ui > div.qc-cmp2-footer.qc-cmp2-footer-overlay.qc-cmp2-footer-scrolled > div > button.css-1wc0q5e > span').click()
except:
    pass


summary_df = stats_summary(driver, "Serie A")
    

driver.close()

Page title: Serie A Summary | WhoScored.com
Page title: Serie A Relegation Playoff Summary | WhoScored.com
Page title: Serie A Team Statistics | WhoScored.com


In [75]:
summary_df

,Team,Goals,Shots pg,yellow,Discipline,Possession%,Pass%,AerialsWon,Rating
0,1. Napoli,77,16.2,47,1,62.1,87.8,12.8,6.81
1,2. Lazio,60,11.5,87,2,52.2,85.4,10.7,6.74
2,3. Inter,71,16.6,62,3,55.7,85.6,13.3,6.71
3,4. Juventus,56,14.1,72,6,49.4,83.6,12.8,6.71
4,5. AC Milan,64,14.5,86,2,55.6,82.6,14.8,6.70
5,6. Atalanta,66,13.5,81,3,49.9,80.6,15.2,6.67
6,7. Roma,50,13.1,78,4,48.1,81.3,14.4,6.64
7,8. Bologna,53,12.8,81,3,55.0,83.0,9.8,6.60
8,9. Fiorentina,53,15.9,89,3,56.4,82.3,15.5,6.60
9,10. Udinese,47,13.3,86,3,47.5,81.8,12.7,6.58


In [7]:
#estadisticas summary table para liga con 18 equipos temporada 22/23
def stats_summary2(driver, league):
    
    #object of ActionChains
    a = ActionChains(driver)
    #identify element: "Leagues"
    m = driver.find_element_by_link_text(league)
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    print("Page title: " + driver.title)
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(9.0, 12.0))
    
    #change to a different season
    #object of ActionChains
    a = ActionChains(driver)
    #identify foldable element: "season"
    m = driver.find_element(By.ID, "seasons")
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    #click on a specific season 
    sleep(5)
    try:
        driver.find_element(By.CSS_SELECTOR, '#seasons > option:nth-child(2)').click()
    except:
        pass
     
    
    print("Page title: " + driver.title)
    
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(9.0, 12.0))
    
    a = ActionChains(driver)
    #identify element: "Leagues"
    s = driver.find_element_by_link_text("Team Statistics")
    #hover over element to chose the specific league
    a.move_to_element(s).click().perform()
    print("Page title: " + driver.title)
    
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(13.0, 15.0))
    
    table_data = []
    
    #summary table for team statistics
    cols = [e.text for e in driver.find_elements(By.TAG_NAME, 'th')[0:8]]
    cols.insert(3, "yellow")
    
    team = [i.find_element(By.TAG_NAME, 'a').text for i in driver.find_elements(By.CSS_SELECTOR, 'td.col12-lg-2.col12-m-3.col12-s-4.col12-xs-5.grid-abs.overflow-text')]
    link = [i.find_element(By.TAG_NAME, 'a').get_attribute('href') for i in driver.find_elements(By.CSS_SELECTOR, 'td.col12-lg-2.col12-m-3.col12-s-4.col12-xs-5.grid-abs.overflow-text')]

    goal = [i.text for i in driver.find_elements(By.CLASS_NAME, 'goal') if i.text.isdigit()]
    shotspergame = [i.text for i in driver.find_elements(By.CLASS_NAME, 'shotsPerGame')[1:]]
    yellow = [i.text for i in driver.find_elements(By.CLASS_NAME, 'yellow-card-box') if i.text.isdigit()]
    red = [i.text for i in driver.find_elements(By.CLASS_NAME, 'red-card-box') if i.text.isdigit()]
    possession = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.possession')]
    passes = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.passSuccess')]
    aerialswon = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.aerialWonPerGame')]
    rating = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.sorted')[0:18]]
    
    table_data.append([team, goal, shotspergame, yellow, red, possession, passes, aerialswon, rating]) 
    
    table = {col : data for col, data in zip(cols, table_data[0])}
    summary_df = pd.DataFrame(table)
    sleep(3)
    return summary_df
   

In [8]:
#estadisticas goal types para ligas con 20 equipos temporada 22/23
def stats_goaltypes(driver, league)->pd.DataFrame:
    #object of ActionChains
    a = ActionChains(driver)
    #identify element: "Leagues"
    m = driver.find_element_by_link_text(league)
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    print("Page title: " + driver.title)
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(6.0, 10.0))
    
    #change to a different season
    #object of ActionChains
    a = ActionChains(driver)
    #identify foldable element: "season"
    m = driver.find_element(By.ID, "seasons")
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    #click on a specific season 
    sleep(5)
    try:
        driver.find_element(By.CSS_SELECTOR, '#seasons > option:nth-child(2)').click()
    except:
        pass
     
    print("Page title: " + driver.title)
    
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(9.0, 12.0))
    
    if league == "Serie A":
        #change to the correct championship
        #object of ActionChains
        a = ActionChains(driver)
        #identify foldable element: "season"
        m = driver.find_element(By.ID, "stages")
        #hover over element to chose the specific league
        a.move_to_element(m).click().perform()
        #click on a specific season 
        sleep(5)
        try:
            driver.find_element(By.XPATH, '//*[@id="stages"]/option[1]').click()
            
        except:
            pass
    else:
        pass
    print("Page title: " + driver.title)

    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(13.0, 15.0))
    
    a = ActionChains(driver)
    #identify element: "Leagues"
    s = driver.find_element_by_link_text("Team Statistics")
    #hover over element to chose the specific league
    a.move_to_element(s).click().perform()
    print("Page title: " + driver.title)
    #Goal types table team statistics
    cols2 = [e.text for e in driver.find_elements(By.TAG_NAME, 'th')[9:15]]
    #colnames = [cols[i] for i in [2,3,4,5,6,7,8,9,10,12,13,14,15,-1]]
    team2 = [i.find_element(By.TAG_NAME, 'a').text for i in driver.find_elements(By.CSS_SELECTOR, 'td.tn')[0:20]]
    open_play = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.op.sorted')]
    counter_attack = [i.text for i in driver.find_elements(By.CLASS_NAME, 'fb') if i.text.isdigit()]
    set_piece = [i.text for i in driver.find_elements(By.CLASS_NAME, 'sp') if i.text.isdigit()]
    penalty = [i.text for i in driver.find_elements(By.CLASS_NAME, 'pen') if i.text.isdigit()]
    own_goal = [i.text for i in driver.find_elements(By.CLASS_NAME, 'og') if i.text.isdigit()]
    table_data2 = []
    table_data2.append([team2, open_play, counter_attack, set_piece, penalty, own_goal])
    
    table = {col : data for col, data in zip(cols2, table_data2[0])}
    goaltypes_df = pd.DataFrame(table)
    
    return goaltypes_df
    

In [9]:
#estadisticas goal types para ligas con 18 equipos temporada 22/23
def stats_goaltypes2(driver, league)->pd.DataFrame:
    #object of ActionChains
    a = ActionChains(driver)
    #identify element: "Leagues"
    m = driver.find_element_by_link_text(league)
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    print("Page title: " + driver.title)
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(6.0, 10.0))
    
    #change to a different season
    #object of ActionChains
    a = ActionChains(driver)
    #identify foldable element: "season"
    m = driver.find_element(By.ID, "seasons")
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    #click on a specific season 
    sleep(5)
    try:
        driver.find_element(By.CSS_SELECTOR, '#seasons > option:nth-child(2)').click()
    except:
        pass
     
    
    print("Page title: " + driver.title)
    
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(9.0, 12.0))
    
    
    a = ActionChains(driver)
    #identify element: "Leagues"
    s = driver.find_element_by_link_text("Team Statistics")
    #hover over element to chose the specific league
    a.move_to_element(s).click().perform()
    print("Page title: " + driver.title)
    #Goal types table team statistics
    cols2 = [e.text for e in driver.find_elements(By.TAG_NAME, 'th')[9:15]]
    #colnames = [cols[i] for i in [2,3,4,5,6,7,8,9,10,12,13,14,15,-1]]
    team2 = [i.find_element(By.TAG_NAME, 'a').text for i in driver.find_elements(By.CSS_SELECTOR, 'td.tn')[0:18]]
    open_play = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.op.sorted')]
    counter_attack = [i.text for i in driver.find_elements(By.CLASS_NAME, 'fb') if i.text.isdigit()]
    set_piece = [i.text for i in driver.find_elements(By.CLASS_NAME, 'sp') if i.text.isdigit()]
    penalty = [i.text for i in driver.find_elements(By.CLASS_NAME, 'pen') if i.text.isdigit()]
    own_goal = [i.text for i in driver.find_elements(By.CLASS_NAME, 'og') if i.text.isdigit()]
    table_data2 = []
    table_data2.append([team2, open_play, counter_attack, set_piece, penalty, own_goal])
    
    table = {col : data for col, data in zip(cols2, table_data2[0])}
    goaltypes_df = pd.DataFrame(table)
    
    return goaltypes_df
    

In [10]:
#estadisticas para attack side de ligas con 20 equipos temporada 22/23
def stats_attackside(driver, league)->pd.DataFrame:
  
    #object of ActionChains
    a = ActionChains(driver)
    #identify element: "Leagues"
    m = driver.find_element_by_link_text(league)
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    print("Page title: " + driver.title)
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(6.0, 10.0))
    
    #change to a different season
    #object of ActionChains
    a = ActionChains(driver)
    #identify foldable element: "season"
    m = driver.find_element(By.ID, "seasons")
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    #click on a specific season 
    sleep(5)
    try:
        driver.find_element(By.CSS_SELECTOR, '#seasons > option:nth-child(2)').click()
    except:
        pass
    
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(13.0, 15.0))
    
    if league == "Serie A":
        #change to the correct championship
        #object of ActionChains
        a = ActionChains(driver)
        #identify foldable element: "season"
        m = driver.find_element(By.ID, "stages")
        #hover over element to chose the specific league
        a.move_to_element(m).click().perform()
        #click on a specific season 
        sleep(5)
        try:
            driver.find_element(By.XPATH, '//*[@id="stages"]/option[1]').click()
            
        except:
            pass
    else:
        pass

    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(13.0, 15.0))
    print("Page title: " + driver.title)
    
    
    a = ActionChains(driver)
    #identify element: "Leagues"
    s = driver.find_element_by_link_text("Team Statistics")
    #hover over element to chose the specific league
    a.move_to_element(s).click().perform()
    print("Page title: " + driver.title)
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(13.0, 15.0))
    
    #attack side table team statistics 
    cols3 = [e.text for e in driver.find_elements(By.TAG_NAME, 'th')[28:32]]
    
    team3 = [i.find_element(By.TAG_NAME, 'a').text for i in driver.find_elements(By.CSS_SELECTOR, 'td.tn')[20:40]]
    overall = [i.text for i in driver.find_elements(By.CLASS_NAME, 'stat-value')[20:]]
    left = overall[::3]
    middle = overall[1::3]
    right = overall[2::3]
    table_data3 = []
    table_data3 = ([team3, left, middle, right])
    
    table = {col : data for col, data in zip(cols3, table_data3)}
    attackside_df = pd.DataFrame(table)
    
    return attackside_df
    

In [11]:
#estadisticas para attack side de ligas con 18 equipos temporada 22/23
def stats_attackside2(driver, league)->pd.DataFrame:
   
    #object of ActionChains
    a = ActionChains(driver)
    #identify element: "Leagues"
    m = driver.find_element_by_link_text(league)
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    print("Page title: " + driver.title)
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(6.0, 10.0))
    
    #change to a different season
    #object of ActionChains
    a = ActionChains(driver)
    #identify foldable element: "season"
    m = driver.find_element(By.ID, "seasons")
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    #click on a specific season 
    sleep(5)
    try:
        driver.find_element(By.CSS_SELECTOR, '#seasons > option:nth-child(2)').click()
    except:
        pass
     
    
    print("Page title: " + driver.title)
    
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(9.0, 12.0))
    
    a = ActionChains(driver)
    #identify element: "Leagues"
    s = driver.find_element_by_link_text("Team Statistics")
    #hover over element to chose the specific league
    a.move_to_element(s).click().perform()
    print("Page title: " + driver.title)
    
    #attack side table team statistics 
    cols3 = [e.text for e in driver.find_elements(By.TAG_NAME, 'th')[28:32]]
    
    team3 = [i.find_element(By.TAG_NAME, 'a').text for i in driver.find_elements(By.CSS_SELECTOR, 'td.tn')[18:36]]
    overall = [i.text for i in driver.find_elements(By.CLASS_NAME, 'stat-value')[18:]]
    left = overall[::3]
    middle = overall[1::3]
    right = overall[2::3]
    table_data3 = []
    table_data3 = ([team3, left, middle, right])
    
    table = {col : data for col, data in zip(cols3, table_data3)}
    attackside_df = pd.DataFrame(table)
    
    return attackside_df
    

In [12]:
#estadisticas para action zones ligas con 20 equipos temporada 22/23
def stats_actionzones(driver, league)->pd.DataFrame:
  
    sleep(2)
    #object of ActionChains
    a = ActionChains(driver)
    #identify element: "Leagues"
    m = driver.find_element_by_link_text(league)
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    print("Page title: " + driver.title)
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(6.0, 10.0))
    
    #change to a different season
    #object of ActionChains
    a = ActionChains(driver)
    #identify foldable element: "season"
    m = driver.find_element(By.ID, "seasons")
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    #click on a specific season 
    sleep(5)
    try:
        driver.find_element(By.CSS_SELECTOR, '#seasons > option:nth-child(2)').click()
    except:
        pass
     
    print("Page title: " + driver.title)
    
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(9.0, 12.0))
    if league == "Serie A":
        #change to the correct championship
        #object of ActionChains
        a = ActionChains(driver)
        #identify foldable element: "season"
        m = driver.find_element(By.ID, "stages")
        #hover over element to chose the specific league
        a.move_to_element(m).click().perform()
        #click on a specific season 
        sleep(5)
        try:
            driver.find_element(By.XPATH, '//*[@id="stages"]/option[1]').click()
            
        except:
            pass
    else:
        pass
    print("Page title: " + driver.title)
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(13.0, 15.0))
    
    a = ActionChains(driver)
    #identify element: "Leagues"
    s = driver.find_element_by_link_text("Team Statistics")
    #hover over element to chose the specific league
    a.move_to_element(s).click().perform()
    print("Page title: " + driver.title)    
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(10.0, 15.0))
    
    
    try:
        driver.find_element(By.CSS_SELECTOR, '#stage-pitch-stats-options > li:nth-child(4) > a').click()
    except:
        pass

    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(10.0, 15.0))
    
    #action zones table team statistics 
    cols4 = [e.text for e in driver.find_elements(By.TAG_NAME, 'th')[-4:]]
    
    team4 = [i.find_element(By.TAG_NAME, 'a').text for i in driver.find_elements(By.CSS_SELECTOR, 'td.tn')[-20:]]
    overall = [i.text for i in driver.find_elements(By.CLASS_NAME, 'stat-value')[-60:]]
    own = overall[::3]
    middle = overall[1::3]
    opp = overall[2::3]
    table_data4 = []
    table_data4 = ([team4, own, middle, opp])
    
    table = {col : data for col, data in zip(cols4, table_data4)}
    actionzones_df = pd.DataFrame(table)
    
    return actionzones_df


In [13]:
#estadisticas para action zones ligas con 18 equipos temporada 22/23
def stats_actionzones2(driver, league)->pd.DataFrame:
   
    sleep(2)
    #object of ActionChains
    a = ActionChains(driver)
    #identify element: "Leagues"
    m = driver.find_element_by_link_text(league)
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    print("Page title: " + driver.title)
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(6.0, 10.0))
    
    #change to a different season
    #object of ActionChains
    a = ActionChains(driver)
    #identify foldable element: "season"
    m = driver.find_element(By.ID, "seasons")
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    #click on a specific season 
    sleep(5)
    try:
        driver.find_element(By.CSS_SELECTOR, '#seasons > option:nth-child(2)').click()
    except:
        pass
     
    
    print("Page title: " + driver.title)
    
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(9.0, 12.0))
    
    
    a = ActionChains(driver)
    #identify element: "Leagues"
    s = driver.find_element_by_link_text("Team Statistics")
    #hover over element to chose the specific league
    a.move_to_element(s).click().perform()
    print("Page title: " + driver.title)    
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(10.0, 15.0))
    
    #find and click on action zones
    try:
        driver.find_element(By.CSS_SELECTOR, '#stage-pitch-stats-options > li:nth-child(4) > a').click()
    except:
        pass

    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(10.0, 15.0))
    
    #action zones table team statistics 
    cols4 = [e.text for e in driver.find_elements(By.TAG_NAME, 'th')[-4:]]
    
    team4 = [i.find_element(By.TAG_NAME, 'a').text for i in driver.find_elements(By.CSS_SELECTOR, 'td.tn')[-18:]]
    overall = [i.text for i in driver.find_elements(By.CLASS_NAME, 'stat-value')[-54:]]
    own = overall[::3]
    middle = overall[1::3]
    opp = overall[2::3]
    table_data4 = []
    table_data4 = ([team4, own, middle, opp])
    
    table = {col : data for col, data in zip(cols4, table_data4)}
    actionzones_df = pd.DataFrame(table)
    
    return actionzones_df


In [14]:
#estadisticas para defensive ligas con 20 equipos temporada 22/23
def stats_defensive(driver, league)->pd.DataFrame:
    
    #object of ActionChains
    a = ActionChains(driver)
    #identify element: "Leagues"
    m = driver.find_element_by_link_text(league)
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    print("Page title: " + driver.title)
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(6.0, 10.0))
    
    #change to a different season
    #object of ActionChains
    a = ActionChains(driver)
    #identify foldable element: "season"
    m = driver.find_element(By.ID, "seasons")
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    #click on a specific season 
    sleep(5)
    try:
        driver.find_element(By.CSS_SELECTOR, '#seasons > option:nth-child(2)').click()
    except:
        pass
    
    print("Page title: " + driver.title)
    
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(9.0, 12.0))
    
    if league == "Serie A":
        #change to the correct championship
        #object of ActionChains
        a = ActionChains(driver)
        #identify foldable element: "season"
        m = driver.find_element(By.ID, "stages")
        #hover over element to chose the specific league
        a.move_to_element(m).click().perform()
        #click on a specific season 
        sleep(5)
        try:
            driver.find_element(By.XPATH, '//*[@id="stages"]/option[1]').click()
            
        except:
            pass
    else:
        pass

    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(13.0, 15.0))
    print("Page title: " + driver.title)
    
    a = ActionChains(driver)
    #identify element: "Leagues"
    s = driver.find_element_by_link_text("Team Statistics")
    #hover over element to chose the specific league
    a.move_to_element(s).click().perform()
    print("Page title: " + driver.title)
    
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(8.0, 10.0))
    
    a = ActionChains(driver)
    #identify element: "Leagues"
    s = driver.find_element_by_link_text("Defensive")
    #hover over element to chose the specific league
    a.move_to_element(s).click().perform()
    print("Page title: " + driver.title)
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(8.0, 10.0))
    
    #table defensive statistics team stats
    cols6 = [e.text for e in driver.find_elements(By.TAG_NAME, 'th')[8:14]]
    
    team6 = [i.find_element(By.TAG_NAME, 'a').text for i in driver.find_elements(By.CSS_SELECTOR, 'td.col12-lg-2.col12-m-3.col12-s-4.col12-xs-5.grid-abs.overflow-text')[20:]]
    shotsconcided = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.shotsConcededPerGame')]
    tackles = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.tacklePerGame.sorted')]
    intercept = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.interceptionPerGame')]
    fouls = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.foulsPerGame')]
    offsidegiven = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.offsideGivenPerGame')]
    table_data6 = []
    table_data6 = ([team6, shotsconcided, tackles, intercept, fouls, offsidegiven])
     
    table = {col : data for col, data in zip(cols6, table_data6)}
    defensive_df = pd.DataFrame(table)
    
    return defensive_df
    

In [15]:
#estadisticas para defensive ligas con 18 equipos temporada 22/23
def stats_defensive2(driver, league)->pd.DataFrame:
   
    #object of ActionChains
    a = ActionChains(driver)
    #identify element: "Leagues"
    m = driver.find_element_by_link_text(league)
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    print("Page title: " + driver.title)
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(6.0, 10.0))
    
    #change to a different season
    #object of ActionChains
    a = ActionChains(driver)
    #identify foldable element: "season"
    m = driver.find_element(By.ID, "seasons")
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    #click on a specific season 
    sleep(5)
    try:
        driver.find_element(By.CSS_SELECTOR, '#seasons > option:nth-child(2)').click()
    except:
        pass
     
    
    print("Page title: " + driver.title)
    
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(9.0, 12.0))
    
    
    a = ActionChains(driver)
    #identify element: "Leagues"
    s = driver.find_element_by_link_text("Team Statistics")
    #hover over element to chose the specific league
    a.move_to_element(s).click().perform()
    print("Page title: " + driver.title)
    
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(8.0, 10.0))
    
    a = ActionChains(driver)
    #identify element: "Leagues"
    s = driver.find_element_by_link_text("Defensive")
    #hover over element to chose the specific league
    a.move_to_element(s).click().perform()
    print("Page title: " + driver.title)
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(8.0, 10.0))
    
    #table defensive statistics team stats
    cols6 = [e.text for e in driver.find_elements(By.TAG_NAME, 'th')[8:14]]
    
    team6 = [i.find_element(By.TAG_NAME, 'a').text for i in driver.find_elements(By.CSS_SELECTOR, 'td.col12-lg-2.col12-m-3.col12-s-4.col12-xs-5.grid-abs.overflow-text')[18:]]
    shotsconcided = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.shotsConcededPerGame')]
    tackles = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.tacklePerGame.sorted')]
    intercept = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.interceptionPerGame')]
    fouls = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.foulsPerGame')]
    offsidegiven = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.offsideGivenPerGame')]
    table_data6 = []
    table_data6 = ([team6, shotsconcided, tackles, intercept, fouls, offsidegiven])
     
    table = {col : data for col, data in zip(cols6, table_data6)}
    defensive_df = pd.DataFrame(table)
    
    return defensive_df
    

In [34]:
#estadisticas para offensive para ligas con 20 equipos temporada 22/23
def stats_offensive(driver, league)->pd.DataFrame:
    
    #object of ActionChains
    a = ActionChains(driver)
    #identify element: "Leagues"
    m = driver.find_element_by_link_text(league)
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    print("Page title: " + driver.title)
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(6.0, 10.0))
    
    #change to a different season
    #object of ActionChains
    a = ActionChains(driver)
    #identify foldable element: "season"
    m = driver.find_element(By.ID, "seasons")
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    #click on a specific season 
    sleep(5)
    try:
        driver.find_element(By.CSS_SELECTOR, '#seasons > option:nth-child(2)').click()
    except:
        pass
     
    
    print("Page title: " + driver.title)
    
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(9.0, 12.0))
    #change to the right championship
    
    if league == "Serie A":
        #change to the correct championship
        #object of ActionChains
        a = ActionChains(driver)
        #identify foldable element: "season"
        m = driver.find_element(By.ID, "stages")
        #hover over element to chose the specific league
        a.move_to_element(m).click().perform()
        #click on a specific season 
        sleep(5)
        try:
            driver.find_element(By.XPATH, '//*[@id="stages"]/option[1]').click()
            
        except:
            pass
    else:
        pass

    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(13.0, 15.0))
    print("Page title: " + driver.title)
    
    a = ActionChains(driver)
    #identify element: "Leagues"
    s = driver.find_element_by_link_text("Team Statistics")
    #hover over element to chose the specific league
    a.move_to_element(s).click().perform()
    print("Page title: " + driver.title)
    
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(14.0, 16.0))
    
    a = ActionChains(driver)
    #identify element: "Leagues"
    s = driver.find_element_by_link_text("Offensive")
    #hover over element to chose the specific league
    a.move_to_element(s).click().perform()
    print("Page title: " + driver.title)
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(12.0, 15.0))
    
    #table ofensive statistics team stats
    cols = [e.text for e in driver.find_elements(By.TAG_NAME, 'th')[8:13]]
    team = [i.find_element(By.TAG_NAME, 'a').text for i in driver.find_elements(By.CSS_SELECTOR, 'td.col12-lg-2.col12-m-3.col12-s-4.col12-xs-5.grid-abs.overflow-text')[20:40]]
    
    shots = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.shotsPerGame.sorted')]
    shotsont = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.shotOnTargetPerGame')]
    dribble = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.dribbleWonPerGame')]
    foulsgiven = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.foulGivenPerGame')]
    table_data = []
    table_data = ([team, shots, shotsont, dribble, foulsgiven])
    
    table = {col : data for col, data in zip(cols, table_data)}
    offensive_df = pd.DataFrame(table)
    
    return offensive_df

In [17]:
#estadisticas para offensive para ligas con 18 equipos temporada 22/23
def stats_offensive2(driver, league)->pd.DataFrame:
    
    #object of ActionChains
    a = ActionChains(driver)
    #identify element: "Leagues"
    m = driver.find_element_by_link_text(league)
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    print("Page title: " + driver.title)
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(6.0, 10.0))
    
    #change to a different season
    #object of ActionChains
    a = ActionChains(driver)
    #identify foldable element: "season"
    m = driver.find_element(By.ID, "seasons")
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    #click on a specific season 
    sleep(5)
    try:
        driver.find_element(By.CSS_SELECTOR, '#seasons > option:nth-child(2)').click()
    except:
        pass
     
    
    print("Page title: " + driver.title)
    
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(9.0, 12.0))
    
    
    a = ActionChains(driver)
    #identify element: "Leagues"
    s = driver.find_element_by_link_text("Team Statistics")
    #hover over element to chose the specific league
    a.move_to_element(s).click().perform()
    print("Page title: " + driver.title)
    
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(8.0, 10.0))
    
    a = ActionChains(driver)
    #identify element: "Leagues"
    s = driver.find_element_by_link_text("Offensive")
    #hover over element to chose the specific league
    a.move_to_element(s).click().perform()
    print("Page title: " + driver.title)
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(10.0, 15.0))
    
    #table ofensive statistics team stats
    cols = [e.text for e in driver.find_elements(By.TAG_NAME, 'th')[8:13]]
    team = [i.find_element(By.TAG_NAME, 'a').text for i in driver.find_elements(By.CSS_SELECTOR, 'td.col12-lg-2.col12-m-3.col12-s-4.col12-xs-5.grid-abs.overflow-text')[18:36]]
    
    shots = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.shotsPerGame.sorted')]
    shotsont = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.shotOnTargetPerGame')]
    dribble = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.dribbleWonPerGame')]
    foulsgiven = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.foulGivenPerGame')]
    table_data = []
    table_data = ([team, shots, shotsont, dribble, foulsgiven])
    
    table = {col : data for col, data in zip(cols, table_data)}
    offensive_df = pd.DataFrame(table)
    
    return offensive_df

In [18]:
#estadisticas detailed table ligas con 20 equipos temporada 22/23
def stats_detailed(driver, league):
    
    #object of ActionChains
    a = ActionChains(driver)
    #identify element: "Leagues"
    m = driver.find_element_by_link_text(league)
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    print("Page title: " + driver.title)
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(6.0, 10.0))
    
    #change to a different season
    #object of ActionChains
    a = ActionChains(driver)
    #identify foldable element: "season"
    m = driver.find_element(By.ID, "seasons")
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    #click on a specific season 
    sleep(5)
    try:
        driver.find_element(By.CSS_SELECTOR, '#seasons > option:nth-child(2)').click()
    except:
        pass
     
    
    print("Page title: " + driver.title)
    
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(9.0, 12.0))
    
    if league == "Serie A":
        #change to the correct championship
        #object of ActionChains
        a = ActionChains(driver)
        #identify foldable element: "season"
        m = driver.find_element(By.ID, "stages")
        #hover over element to chose the specific league
        a.move_to_element(m).click().perform()
        #click on a specific season 
        sleep(5)
        try:
            driver.find_element(By.XPATH, '//*[@id="stages"]/option[1]').click()
            
        except:
            pass
    else:
        pass

    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(13.0, 15.0))
    print("Page title: " + driver.title)
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(9.0, 12.0))
    
    a = ActionChains(driver)
    #identify element: "Leagues"
    s = driver.find_element_by_link_text("Team Statistics")
    #hover over element to chose the specific league
    a.move_to_element(s).click().perform()
    print("Page title: " + driver.title)
    
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(8.0, 10.0))

    a = ActionChains(driver)
    #identify element: "Leagues"
    s = driver.find_element_by_link_text("Detailed")
    #hover over element to chose the specific league
    a.move_to_element(s).click().perform()
    print("Page title: " + driver.title)
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(10.0, 15.0))
    
    #table detailed statistics team stats
    cols6 = [e.text for e in driver.find_elements(By.TAG_NAME, 'th')[8:13]]
    team6 = [i.find_element(By.TAG_NAME, 'a').text for i in driver.find_elements(By.CSS_SELECTOR, 'td.col12-lg-2.col12-m-3.col12-s-4.col12-xs-5.grid-abs.overflow-text')[20:40]]
    
    totalshots = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.shotsTotal')]
    outofbox = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.shotOboxTotal')]
    sixyard = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.shotSixYardBox')]
    penaltyarea = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.shotPenaltyArea')]
    table_data6 = []
    table_data6 = ([team6, totalshots, outofbox, sixyard, penaltyarea])
    
    table = {col : data for col, data in zip(cols6, table_data6)}
    offensive_df = pd.DataFrame(table)
    
    return offensive_df

In [19]:
#estadisticas detailed table ligas con 18 equipos temporada 22/23
def stats_detailed2(driver, league):
    
    #object of ActionChains
    a = ActionChains(driver)
    #identify element: "Leagues"
    m = driver.find_element_by_link_text(league)
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    print("Page title: " + driver.title)
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(6.0, 10.0))
    
    #change to a different season
    #object of ActionChains
    a = ActionChains(driver)
    #identify foldable element: "season"
    m = driver.find_element(By.ID, "seasons")
    #hover over element to chose the specific league
    a.move_to_element(m).click().perform()
    #click on a specific season 
    sleep(5)
    try:
        driver.find_element(By.CSS_SELECTOR, '#seasons > option:nth-child(2)').click()
    except:
        pass
     
    
    print("Page title: " + driver.title)
    
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(9.0, 12.0))
    
    
    a = ActionChains(driver)
    #identify element: "Leagues"
    s = driver.find_element_by_link_text("Team Statistics")
    #hover over element to chose the specific league
    a.move_to_element(s).click().perform()
    print("Page title: " + driver.title)
    
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(8.0, 10.0))

    a = ActionChains(driver)
    #identify element: "Leagues"
    s = driver.find_element_by_link_text("Detailed")
    #hover over element to chose the specific league
    a.move_to_element(s).click().perform()
    print("Page title: " + driver.title)
    #tiempo random entre 6 y 10 segundos
    sleep(random.uniform(10.0, 15.0))
    
    #table detailed statistics team stats
    cols = [e.text for e in driver.find_elements(By.TAG_NAME, 'th')[8:13]]
    team = [i.find_element(By.TAG_NAME, 'a').text for i in driver.find_elements(By.CSS_SELECTOR, 'td.col12-lg-2.col12-m-3.col12-s-4.col12-xs-5.grid-abs.overflow-text')[18:36]]
    
    totalshots = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.shotsTotal')]
    outofbox = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.shotOboxTotal')]
    sixyard = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.shotSixYardBox')]
    penaltyarea = [i.text for i in driver.find_elements(By.CSS_SELECTOR, 'td.shotPenaltyArea')]
    table_data = []
    table_data = ([team, totalshots, outofbox, sixyard, penaltyarea])
    
    table = {col : data for col, data in zip(cols, table_data)}
    offensive_df = pd.DataFrame(table)
    
    return offensive_df

In [20]:
# guardar tablas de las ligas para temporada 22/23
def save_summary_pd(summary_df:pd.DataFrame, league:str)->None:

    summary_df.to_csv(f'{league}_summary_22_23.csv', index=False)

In [21]:
# guardar tablas de las ligas para temporada 22/23
def save_goaltypes_pd(goaltypes_df:pd.DataFrame, league:str)->None:

    goaltypes_df.to_csv(f'{league}_goaltypes_22_23.csv', index=False)

In [22]:
# guardar tablas de las ligas para temporada 22/23
def save_attackside_pd(attackside_df:pd.DataFrame, league:str)->None:

    attackside_df.to_csv(f'{league}_attackside_22_23.csv', index=False)

In [23]:
# guardar tablas de las ligas para temporada 22/23
def save_actionzones_pd(actionzones_df:pd.DataFrame, league:str)->None:

    actionzones_df.to_csv(f'{league}_actionzones_22_23.csv', index=False)

In [24]:
# guardar tablas de las ligas para temporada 22/23
def save_defensive_pd(defensive_df:pd.DataFrame, league:str)->None:

    defensive_df.to_csv(f'{league}_defensive_22_23.csv', index=False)

In [25]:
# guardar tablas de las ligas para temporada 22/23
def save_offensive_pd(offensive_df:pd.DataFrame, league:str)->None:

    offensive_df.to_csv(f'{league}_offensive_22_23.csv', index=False)

In [26]:
# guardar tablas de las ligas para temporada 22/23
def save_detailed_pd(detailed_df:pd.DataFrame, league:str)->None:

    detailed_df.to_csv(f'{league}_detailed_22_23.csv', index=False)

In [57]:
#temporada summary leagues with 18 teams 22/23
start = datetime.now()

chrome_driver = "./chromedriver.exe"
driver = webdriver.Chrome(executable_path = chrome_driver)
driver.get("https://www.whoscored.com/")
try:
    driver.find_element(By.CSS_SELECTOR, '#qc-cmp2-ui > div.qc-cmp2-footer.qc-cmp2-footer-overlay.qc-cmp2-footer-scrolled > div > button.css-1wc0q5e > span').click()
except:
    pass

leagues18 = ["Bundesliga"]


errores = 0
for l in leagues18:
    try:
        summary_df = stats_summary2(driver, l)
        save_summary_pd(summary_df, l)
    except:
        print(f'Error al crear la tabla de {l}')
        errores += 1


driver.quit()

end = datetime.now()

print(f'Proceso finalizado en {end - start} con {errores} errores')

Page title: Bundesliga Summary | WhoScored.com
Page title: Bundesliga Summary | WhoScored.com
Page title: Bundesliga Team Statistics | WhoScored.com
Proceso finalizado en 0:02:04.219814 con 0 errores


In [58]:
#goaltypes leagues with 18 teams temporada 22/23
start = datetime.now()

chrome_driver = "./chromedriver.exe"
driver = webdriver.Chrome(executable_path = chrome_driver)
driver.get("https://www.whoscored.com/")
try:
    driver.find_element(By.CSS_SELECTOR, '#qc-cmp2-ui > div.qc-cmp2-footer.qc-cmp2-footer-overlay.qc-cmp2-footer-scrolled > div > button.css-1wc0q5e > span').click()
except:
    pass

leagues18 = ["Bundesliga"]


errores = 0
for l in leagues18:
    try:
        goaltypes_df = stats_goaltypes2(driver, l)
        save_goaltypes_pd(goaltypes_df, l)
    except:
        print(f'Error al crear la tabla de {l}')
        errores += 1


driver.quit()

end = datetime.now()

print(f'Proceso finalizado en {end - start} con {errores} errores')

Page title: Bundesliga Summary | WhoScored.com
Page title: Bundesliga Summary | WhoScored.com
Page title: Bundesliga Team Statistics | WhoScored.com
Proceso finalizado en 0:02:04.953651 con 0 errores


In [59]:
#attackside leagues with 18 teams temporada 22/23
start = datetime.now()

chrome_driver = "./chromedriver.exe"
driver = webdriver.Chrome(executable_path = chrome_driver)
driver.get("https://www.whoscored.com/")
try:
    driver.find_element(By.CSS_SELECTOR, '#qc-cmp2-ui > div.qc-cmp2-footer.qc-cmp2-footer-overlay.qc-cmp2-footer-scrolled > div > button.css-1wc0q5e > span').click()
except:
    pass

leagues18 = ["Bundesliga"]


errores = 0
for l in leagues18:
    try:
        attackside_df = stats_attackside2(driver, l)
        save_attackside_pd(attackside_df, l)
    except:
        print(f'Error al crear la tabla de {l}')
        errores += 1


driver.quit()

end = datetime.now()

print(f'Proceso finalizado en {end - start} con {errores} errores')

Page title: Bundesliga Summary | WhoScored.com
Page title: Bundesliga Summary | WhoScored.com
Page title: Bundesliga Team Statistics | WhoScored.com
Proceso finalizado en 0:01:05.354692 con 0 errores


In [60]:
#actionzones leagues with 18 teams temporada 22/23
start = datetime.now()

chrome_driver = "./chromedriver.exe"
driver = webdriver.Chrome(executable_path = chrome_driver)
driver.get("https://www.whoscored.com/")
try:
    driver.find_element(By.CSS_SELECTOR, '#qc-cmp2-ui > div.qc-cmp2-footer.qc-cmp2-footer-overlay.qc-cmp2-footer-scrolled > div > button.css-1wc0q5e > span').click()
except:
    pass

leagues18 = ["Bundesliga"]


errores = 0
for l in leagues18:
    try:
        actionzones_df = stats_actionzones2(driver, l)
        save_actionzones_pd(actionzones_df, l)
    except:
        print(f'Error al crear la tabla de {l}')
        errores += 1

driver.quit()

end = datetime.now()

print(f'Proceso finalizado en {end - start} con {errores} errores')

Page title: Bundesliga Summary | WhoScored.com
Page title: Bundesliga Summary | WhoScored.com
Page title: Bundesliga Team Statistics | WhoScored.com
Proceso finalizado en 0:01:16.501434 con 0 errores


In [61]:
#detailed leagues with 18 teams temporada 22/23
start = datetime.now()

chrome_driver = "./chromedriver.exe"
driver = webdriver.Chrome(executable_path = chrome_driver)
driver.get("https://www.whoscored.com/")
try:
    driver.find_element(By.CSS_SELECTOR, '#qc-cmp2-ui > div.qc-cmp2-footer.qc-cmp2-footer-overlay.qc-cmp2-footer-scrolled > div > button.css-1wc0q5e > span').click()
except:
    pass

leagues18 = ["Bundesliga"]


errores = 0
for l in leagues18:
    try:
        detailed_df = stats_detailed2(driver, l)
        save_detailed_pd(detailed_df, l)
    except:
        print(f'Error al crear la tabla de {l}')
        errores += 1


driver.quit()

end = datetime.now()

print(f'Proceso finalizado en {end - start} con {errores} errores')

Page title: Bundesliga Summary | WhoScored.com
Page title: Bundesliga Summary | WhoScored.com
Page title: Bundesliga Team Statistics | WhoScored.com
Page title: Bundesliga Team Statistics | WhoScored.com
Proceso finalizado en 0:01:53.288892 con 0 errores


In [62]:
#defensive leagues with 18 teams temporada 22/23
start = datetime.now()

chrome_driver = "./chromedriver.exe"
driver = webdriver.Chrome(executable_path = chrome_driver)
driver.get("https://www.whoscored.com/")
try:
    driver.find_element(By.CSS_SELECTOR, '#qc-cmp2-ui > div.qc-cmp2-footer.qc-cmp2-footer-overlay.qc-cmp2-footer-scrolled > div > button.css-1wc0q5e > span').click()
except:
    pass

leagues18 = ["Bundesliga"]


errores = 0
for l in leagues18:
    try:
        defensive_df = stats_defensive2(driver, l)
        save_defensive_pd(defensive_df, l)
    except:
        print(f'Error al crear la tabla de {l}')
        errores += 1


driver.quit()

end = datetime.now()

print(f'Proceso finalizado en {end - start} con {errores} errores')

Page title: Bundesliga Summary | WhoScored.com
Page title: Bundesliga Summary | WhoScored.com
Page title: Bundesliga Team Statistics | WhoScored.com
Page title: Bundesliga Team Statistics | WhoScored.com
Proceso finalizado en 0:01:31.023128 con 0 errores


In [63]:
#offensive leagues with 18 teams temporada 22/23
start = datetime.now()

chrome_driver = "./chromedriver.exe"
driver = webdriver.Chrome(executable_path = chrome_driver)
driver.get("https://www.whoscored.com/")
try:
    driver.find_element(By.CSS_SELECTOR, '#qc-cmp2-ui > div.qc-cmp2-footer.qc-cmp2-footer-overlay.qc-cmp2-footer-scrolled > div > button.css-1wc0q5e > span').click()
except:
    pass

leagues18 = ["Bundesliga"]


errores = 0
for l in leagues18:
    try:
        offensive_df = stats_offensive2(driver, l)
        save_offensive_pd(offensive_df, l)
    except:
        print(f'Error al crear la tabla de {l}')
        errores += 1


driver.quit()

end = datetime.now()

print(f'Proceso finalizado en {end - start} con {errores} errores')

Page title: Bundesliga Summary | WhoScored.com
Page title: Bundesliga Summary | WhoScored.com
Page title: Bundesliga Team Statistics | WhoScored.com
Page title: Bundesliga Team Statistics | WhoScored.com
Proceso finalizado en 0:04:24.685152 con 0 errores


In [27]:
#summary with 20 teams temporada 22/23
start = datetime.now()

chrome_driver = "./chromedriver.exe"
driver = webdriver.Chrome(executable_path = chrome_driver)
driver.get("https://www.whoscored.com/")
try:
    driver.find_element(By.CSS_SELECTOR, '#qc-cmp2-ui > div.qc-cmp2-footer.qc-cmp2-footer-overlay.qc-cmp2-footer-scrolled > div > button.css-1wc0q5e > span').click()
except:
    pass

leagues20 = ["Premier League", "Serie A", "LaLiga", "Ligue 1"]

errores = 0
for l in leagues20:
    try:
        summary_df = stats_summary(driver, l)
        save_summary_pd(summary_df, l)
    except:
        print(f'Error al crear la tabla de {l}')
        errores += 1


driver.quit()

end = datetime.now()

print(f'Proceso finalizado en {end - start} con {errores} errores')

Page title: Premier League Summary | WhoScored.com
Page title: Premier League Summary | WhoScored.com
Page title: Premier League Team Statistics | WhoScored.com
Page title: Serie A Summary | WhoScored.com
Page title: Serie A Relegation Playoff Summary | WhoScored.com
Page title: Serie A Team Statistics | WhoScored.com
Page title: LaLiga Summary | WhoScored.com
Page title: LaLiga Summary | WhoScored.com
Page title: LaLiga Team Statistics | WhoScored.com
Page title: Ligue 1 Summary | WhoScored.com
Page title: Ligue 1 Summary | WhoScored.com
Page title: Ligue 1 Team Statistics | WhoScored.com
Proceso finalizado en 0:06:24.009336 con 0 errores


In [28]:
#goaltypes with 20 teams temporada 22/23
start = datetime.now()

chrome_driver = "./chromedriver.exe"
driver = webdriver.Chrome(executable_path = chrome_driver)
driver.get("https://www.whoscored.com/")
try:
    driver.find_element(By.CSS_SELECTOR, '#qc-cmp2-ui > div.qc-cmp2-footer.qc-cmp2-footer-overlay.qc-cmp2-footer-scrolled > div > button.css-1wc0q5e > span').click()
except:
    pass

leagues20 = ["Premier League", "Serie A", "LaLiga", "Ligue 1"]

errores = 0
for l in leagues20:
    try:
        goaltypes_df = stats_goaltypes(driver, l)
        save_goaltypes_pd(goaltypes_df, l)
    except:
        print(f'Error al crear la tabla de {l}')
        errores += 1


driver.quit()

end = datetime.now()

print(f'Proceso finalizado en {end - start} con {errores} errores')

Page title: Premier League Summary | WhoScored.com
Page title: Premier League Summary | WhoScored.com
Page title: Premier League Summary | WhoScored.com
Page title: Premier League Team Statistics | WhoScored.com
Page title: Serie A Summary | WhoScored.com
Page title: Serie A Relegation Playoff Summary | WhoScored.com
Page title: Serie A Summary | WhoScored.com
Page title: Serie A Team Statistics | WhoScored.com
Page title: LaLiga Summary | WhoScored.com
Page title: LaLiga Summary | WhoScored.com
Page title: LaLiga Summary | WhoScored.com
Page title: LaLiga Team Statistics | WhoScored.com
Page title: Ligue 1 Summary | WhoScored.com
Page title: Ligue 1 Summary | WhoScored.com
Page title: Ligue 1 Summary | WhoScored.com
Page title: Ligue 1 Team Statistics | WhoScored.com
Proceso finalizado en 0:04:50.723464 con 0 errores


In [29]:
#attackside with 20 teams temporada 22/23
start = datetime.now()

chrome_driver = "./chromedriver.exe"
driver = webdriver.Chrome(executable_path = chrome_driver)
driver.get("https://www.whoscored.com/")
try:
    driver.find_element(By.CSS_SELECTOR, '#qc-cmp2-ui > div.qc-cmp2-footer.qc-cmp2-footer-overlay.qc-cmp2-footer-scrolled > div > button.css-1wc0q5e > span').click()
except:
    pass

leagues20 = ["Premier League", "Serie A", "LaLiga", "Ligue 1"]

errores = 0
for l in leagues20:
    try:
        attackside_df = stats_attackside(driver, l)
        save_attackside_pd(attackside_df, l)
    except:
        print(f'Error al crear la tabla de {l}')
        errores += 1


driver.quit()

end = datetime.now()

print(f'Proceso finalizado en {end - start} con {errores} errores')

Page title: Premier League Summary | WhoScored.com
Page title: Premier League Summary | WhoScored.com
Page title: Premier League Team Statistics | WhoScored.com
Page title: Serie A Summary | WhoScored.com
Page title: Serie A Summary | WhoScored.com
Page title: Serie A Team Statistics | WhoScored.com
Page title: LaLiga Summary | WhoScored.com
Page title: LaLiga Summary | WhoScored.com
Page title: LaLiga Team Statistics | WhoScored.com
Page title: Ligue 1 Summary | WhoScored.com
Page title: Ligue 1 Summary | WhoScored.com
Page title: Ligue 1 Team Statistics | WhoScored.com
Proceso finalizado en 0:04:46.323804 con 0 errores


In [30]:
#actionzones with 20 teams temporada 22/23
start = datetime.now()

chrome_driver = "./chromedriver.exe"
driver = webdriver.Chrome(executable_path = chrome_driver)
driver.get("https://www.whoscored.com/")
try:
    driver.find_element(By.CSS_SELECTOR, '#qc-cmp2-ui > div.qc-cmp2-footer.qc-cmp2-footer-overlay.qc-cmp2-footer-scrolled > div > button.css-1wc0q5e > span').click()
except:
    pass

leagues20 = ["Premier League", "Serie A", "LaLiga", "Ligue 1"]

errores = 0
for l in leagues20:
    try:
        actionzones_df = stats_actionzones(driver, l)
        save_actionzones_pd(actionzones_df, l)
    except:
        print(f'Error al crear la tabla de {l}')
        errores += 1


driver.quit()

end = datetime.now()

print(f'Proceso finalizado en {end - start} con {errores} errores')

Page title: Premier League Summary | WhoScored.com
Page title: Premier League Summary | WhoScored.com
Page title: Premier League Summary | WhoScored.com
Page title: Premier League Team Statistics | WhoScored.com
Page title: Serie A Summary | WhoScored.com
Page title: Serie A Relegation Playoff Summary | WhoScored.com
Page title: Serie A Summary | WhoScored.com
Page title: Serie A Team Statistics | WhoScored.com
Page title: LaLiga Summary | WhoScored.com
Page title: LaLiga Summary | WhoScored.com
Page title: LaLiga Summary | WhoScored.com
Page title: LaLiga Team Statistics | WhoScored.com
Page title: Ligue 1 Summary | WhoScored.com
Page title: Ligue 1 Summary | WhoScored.com
Page title: Ligue 1 Summary | WhoScored.com
Page title: Ligue 1 Team Statistics | WhoScored.com
Proceso finalizado en 0:06:31.138405 con 0 errores


In [31]:
#detailed with 20 teams temporada 22/23
start = datetime.now()

chrome_driver = "./chromedriver.exe"
driver = webdriver.Chrome(executable_path = chrome_driver)
driver.get("https://www.whoscored.com/")
try:
    driver.find_element(By.CSS_SELECTOR, '#qc-cmp2-ui > div.qc-cmp2-footer.qc-cmp2-footer-overlay.qc-cmp2-footer-scrolled > div > button.css-1wc0q5e > span').click()
except:
    pass

leagues20 = ["Premier League", "Serie A", "LaLiga", "Ligue 1"]

errores = 0
for l in leagues20:
    try:
        detailed_df = stats_detailed(driver, l)
        save_detailed_pd(detailed_df, l)
    except:
        print(f'Error al crear la tabla de {l}')
        errores += 1


driver.quit()

end = datetime.now()

print(f'Proceso finalizado en {end - start} con {errores} errores')

Page title: Premier League Summary | WhoScored.com
Page title: Premier League Summary | WhoScored.com
Page title: Premier League Summary | WhoScored.com
Page title: Premier League Team Statistics | WhoScored.com
Page title: Premier League Team Statistics | WhoScored.com
Page title: Serie A Summary | WhoScored.com
Page title: Serie A Relegation Playoff Summary | WhoScored.com
Page title: Serie A Summary | WhoScored.com
Page title: Serie A Team Statistics | WhoScored.com
Page title: Serie A Team Statistics | WhoScored.com
Page title: LaLiga Summary | WhoScored.com
Page title: LaLiga Summary | WhoScored.com
Page title: LaLiga Summary | WhoScored.com
Page title: LaLiga Team Statistics | WhoScored.com
Page title: LaLiga Team Statistics | WhoScored.com
Page title: Ligue 1 Summary | WhoScored.com
Page title: Ligue 1 Summary | WhoScored.com
Page title: Ligue 1 Summary | WhoScored.com
Page title: Ligue 1 Team Statistics | WhoScored.com
Page title: Ligue 1 Team Statistics | WhoScored.com
Proceso

In [32]:
#defensive with 20 teams temporada 22/23
start = datetime.now()

chrome_driver = "./chromedriver.exe"
driver = webdriver.Chrome(executable_path = chrome_driver)
driver.get("https://www.whoscored.com/")
try:
    driver.find_element(By.CSS_SELECTOR, '#qc-cmp2-ui > div.qc-cmp2-footer.qc-cmp2-footer-overlay.qc-cmp2-footer-scrolled > div > button.css-1wc0q5e > span').click()
except:
    pass

leagues20 = ["Premier League", "Serie A", "LaLiga", "Ligue 1"]

errores = 0
for l in leagues20:
    try:
        defensive_df = stats_defensive(driver, l)
        save_defensive_pd(defensive_df, l)
    except:
        print(f'Error al crear la tabla de {l}')
        errores += 1


driver.quit()

end = datetime.now()

print(f'Proceso finalizado en {end - start} con {errores} errores')

Page title: Premier League Summary | WhoScored.com
Page title: Premier League Summary | WhoScored.com
Page title: Premier League Summary | WhoScored.com
Page title: Premier League Team Statistics | WhoScored.com
Page title: Premier League Team Statistics | WhoScored.com
Page title: Serie A Summary | WhoScored.com
Page title: Serie A Relegation Playoff Summary | WhoScored.com
Page title: Serie A Summary | WhoScored.com
Page title: Serie A Team Statistics | WhoScored.com
Page title: Serie A Team Statistics | WhoScored.com
Page title: LaLiga Summary | WhoScored.com
Page title: LaLiga Summary | WhoScored.com
Page title: LaLiga Summary | WhoScored.com
Page title: LaLiga Team Statistics | WhoScored.com
Page title: LaLiga Team Statistics | WhoScored.com
Page title: Ligue 1 Summary | WhoScored.com
Page title: Ligue 1 Summary | WhoScored.com
Page title: Ligue 1 Summary | WhoScored.com
Page title: Ligue 1 Team Statistics | WhoScored.com
Page title: Ligue 1 Team Statistics | WhoScored.com
Proceso

In [33]:
#offensive with 20 teams temporada 22/23
start = datetime.now()

chrome_driver = "./chromedriver.exe"
driver = webdriver.Chrome(executable_path = chrome_driver)
driver.get("https://www.whoscored.com/")
try:
    driver.find_element(By.CSS_SELECTOR, '#qc-cmp2-ui > div.qc-cmp2-footer.qc-cmp2-footer-overlay.qc-cmp2-footer-scrolled > div > button.css-1wc0q5e > span').click()
except:
    pass

leagues20 = ["Premier League", "Serie A", "LaLiga", "Ligue 1"]

errores = 0
for l in leagues20:
    try:
        offensive_df = stats_offensive(driver, l)
        save_offensive_pd(offensive_df, l)
    except:
        print(f'Error al crear la tabla de {l}')
        errores += 1


driver.quit()

end = datetime.now()

print(f'Proceso finalizado en {end - start} con {errores} errores')

Page title: Premier League Summary | WhoScored.com
Page title: Premier League Summary | WhoScored.com
Page title: Premier League Summary | WhoScored.com
Page title: Premier League Team Statistics | WhoScored.com
Page title: Premier League Team Statistics | WhoScored.com
Page title: Serie A Summary | WhoScored.com
Page title: Serie A Relegation Playoff Summary | WhoScored.com
Page title: Serie A Summary | WhoScored.com
Error al crear la tabla de Serie A
Page title: LaLiga Summary | WhoScored.com
Page title: LaLiga Summary | WhoScored.com
Page title: LaLiga Summary | WhoScored.com
Page title: LaLiga Team Statistics | WhoScored.com
Page title: LaLiga Team Statistics | WhoScored.com
Page title: Ligue 1 Summary | WhoScored.com
Page title: Ligue 1 Summary | WhoScored.com
Page title: Ligue 1 Summary | WhoScored.com
Page title: Ligue 1 Team Statistics | WhoScored.com
Page title: Ligue 1 Team Statistics | WhoScored.com
Proceso finalizado en 0:10:58.162550 con 1 errores


In [36]:
start = datetime.now()

chrome_driver = "./chromedriver.exe"
driver = webdriver.Chrome(executable_path = chrome_driver)
driver.get("https://www.whoscored.com/")
try:
    driver.find_element(By.CSS_SELECTOR, '#qc-cmp2-ui > div.qc-cmp2-footer.qc-cmp2-footer-overlay.qc-cmp2-footer-scrolled > div > button.css-1wc0q5e > span').click()
except:
    pass


offensive_df = stats_offensive(driver, "Serie A")
save_offensive_pd(offensive_df, "Serie A")
    
driver.quit()

end = datetime.now()

print(f'Proceso finalizado en {end - start}')

Page title: Serie A Summary | WhoScored.com
Page title: Serie A Relegation Playoff Summary | WhoScored.com
Page title: Serie A Summary | WhoScored.com
Page title: Serie A Team Statistics | WhoScored.com
Page title: Serie A Team Statistics | WhoScored.com
Proceso finalizado en 0:01:56.659061
